In [2]:
from riotwatcher import LolWatcher, RiotWatcher, ApiError
import time
import pandas as pd
from proj_utils import REGIONS
APIKEY = open("riot.txt", "r").readline()

lolWatcher = LolWatcher(APIKEY)
riotWatcher = RiotWatcher(APIKEY)

REGION = "eun1"
PLATFORM = "EUROPE"
# QUEUE = "RANKED_SOLO_5x5" "RANKED_FLEX_SR" "RANKED_FLEX_TT"

In [4]:
class DataDragonUtility:
    def __init__(self):
        self.currentVersion = lolWatcher.data_dragon.versions_all()[0]
        self.items = lolWatcher.data_dragon.items(version=self.currentVersion)
        
    def itemIdToName(self, id):
        return self.items["data"][str(id)]["name"]
    
def accountByRiotId(gameName, tagLine, platform=PLATFORM):
# {'puuid': 'vEgBphSIP4kLidak6uIjOpxOBdvA6oAC_8hL2hKFm22O0WdqmxCkH-B_ARGTCns6KAkwDjKB5e0lhw', 
# 'gameName': 'BoldogKatica1', 'tagLine': 'EUNE'}
    return riotWatcher.account.by_riot_id(platform, gameName, tagLine)

def summonerByPuuid(puuid, region=REGION):
# {'id': 'cOhTGpoZzlyUAXZ1_17XAmr73Tl1W35ZM4904B-uWoYcbUw', 'accountId': 'WKGBVBYq-6X_SbxGPc0Q0Oj15-9jFLYd53cdPTUCQ1PASA',
# 'puuid': 'vEgBphSIP4kLidak6uIjOpxOBdvA6oAC_8hL2hKFm22O0WdqmxCkH-B_ARGTCns6KAkwDjKB5e0lhw', 'profileIconId': 1638, 'revisionDate': 1717009508223, 'summonerLevel': 501}
    return lolWatcher.summoner.by_puuid(region, puuid)

def matchlistByPuuid(puuid, 
    count=1, 
    region=REGION,
    start = None,
    queue = None,
    type = None,
    start_time = None,
    end_time = None
    ):
# ['EUN1_3603423536', 'EUN1_3603410322', 'EUN1_3603355849']
# type = normal, tournament, ranked, tutorial
    return lolWatcher.match.matchlist_by_puuid(
        region = region,
        puuid = puuid,
        start = start,
        count = count,
        queue = queue,
        type = type,
        start_time = start_time,
        end_time = end_time
        )

def timelineByMatch(matchId, region=REGION):
# https://developer.riotgames.com/apis#match-v5/GET_getTimeline
    return lolWatcher.match.timeline_by_match(region, matchId)

def timelineMatchToEvents(matchTimelineData, include = None):
    eventFrames = [frame["events"] for frame in matchTimelineData["info"]["frames"]]
    events = []
    if include:
        [events.append(event) for frame in eventFrames for event in frame if event["type"] in include]
    else:
        [events.append(event) for frame in eventFrames for event in frame]
    return events


In [7]:
ddu = DataDragonUtility()

me = ("BoldogKatica1", "EUNE")
account = accountByRiotId(me[0], me[1])
summoner = summonerByPuuid(account["puuid"])
matches = matchlistByPuuid(account["puuid"], count=10, type="normal")
lastMatch = matches[0]
matchData = timelineByMatch(lastMatch)

print("puuid: ", account["puuid"])


puuid:  vEgBphSIP4kLidak6uIjOpxOBdvA6oAC_8hL2hKFm22O0WdqmxCkH-B_ARGTCns6KAkwDjKB5e0lhw


In [8]:
matches


['EUN1_3604522903',
 'EUN1_3604518964',
 'EUN1_3604521658',
 'EUN1_3601820950',
 'EUN1_3594803484',
 'EUN1_3593454384',
 'EUN1_3592995925',
 'EUN1_3592488047',
 'EUN1_3592471416',
 'EUN1_3592439807']

In [40]:
### Sample match
# include = ["ITEM_UNDO", "ITEM_PURCHASED", "ITEM_DESTROYED"]
# events = timelineMatchToEvents(matchData, include=include)
# for e in events:
#     if e["type"] == "ITEM_UNDO":
#         print("Player:", e["participantId"], e["type"], ddu.itemIdToName(e["beforeId"]), e["timestamp"])
#     else:
#         print("Player:", e["participantId"], e["type"], ddu.itemIdToName(e["itemId"]), e["timestamp"])

In [8]:
# get summoners from different divisions
DIVS = ["I", "II", "III", "IV"]
TIERS = ["DIAMOND", "EMERALD", "PLATINUM", "GOLD", "SILVER", "BRONZE", "IRON"]
QUEUES = [
          "RANKED_SOLO_5x5", 
          "RANKED_FLEX_SR", 
#          "RANKED_FLEX_TT"
          ]

players = set()

for region in REGIONS:
    for queue in QUEUES:
        ### rate limit: 100 requests every 2 minutes(s)
        time.sleep(120./100.)
        try:
            leagueList = lolWatcher.league.challenger_by_queue(region, queue)
            for playerInfo in leagueList["entries"]:
                entry = (region,
                            queue,
                            "CHALLENGER",
                            "I",
                            leagueList["leagueId"],
                            playerInfo["summonerId"],
                            playerInfo["wins"],
                            playerInfo["losses"],
                            playerInfo["leaguePoints"]
                            )
                players.add(entry)
        except:
            print("Failed: ", region, queue, "challenger")
            
        time.sleep(120./100.)
        
        try:
            leagueList = lolWatcher.league.masters_by_queue(region, queue)
            for playerInfo in leagueList["entries"]:
                entry = (region,
                            queue,
                            "MASTER",
                            "I",
                            leagueList["leagueId"],
                            playerInfo["summonerId"],
                            playerInfo["wins"],
                            playerInfo["losses"],
                            playerInfo["leaguePoints"]
                            )
                players.add(entry)
        except:
            print("Failed: ", region, queue, "masters")

        time.sleep(120./100.)
        
        try:
            leagueList = lolWatcher.league.grandmaster_by_queue(region, queue)
            for playerInfo in leagueList["entries"]:
                entry = (region,
                            queue,
                            "GRANDMASTER",
                            "I",
                            leagueList["leagueId"],
                            playerInfo["summonerId"],
                            playerInfo["wins"],
                            playerInfo["losses"],
                            playerInfo["leaguePoints"]
                            )
                players.add(entry)
        except:
            print("Failed: ", region, queue, "gransmasters")

for region in REGIONS:
    print("Reading:", str(region))
    for queue in QUEUES:
        for tier in TIERS:
            for div in DIVS:
                ### rate limit: 100 requests every 2 minutes(s)
                time.sleep(120./100.)
                try:
                    playerInfoList = lolWatcher.league.entries(region, queue, tier, div, page=5)
                    for playerInfo in playerInfoList:
                        entry = (region,
                                queue,
                                tier,
                                div,
                                playerInfo["leagueId"],
                                playerInfo["summonerId"],
                                playerInfo["wins"],
                                playerInfo["losses"],
                                playerInfo["leaguePoints"]
                                )
                        players.add(entry)
                except:
                    print("Failed: ", region, queue, tier, div)



# save
players = list(players)
playersDf = pd.DataFrame(players, 
                         columns=["region", "queue", 
                                  "tier", "div", 
                                  "leagueId", "summonerId", 
                                  "wins", "losses", 
                                  "leaguePoints"])
playersDf.to_parquet("./playerLeagues.parquet", index=True)

Reading: EUN1
Reading: EUW1
Reading: JP1
Reading: KR
Failed:  KR RANKED_FLEX_SR GOLD IV
Failed:  KR RANKED_FLEX_SR SILVER I
Failed:  KR RANKED_FLEX_SR SILVER II
Reading: NA1
Reading: OC1
Reading: RU
Reading: TR1


In [2]:
playersDf = pd.read_parquet("./playerLeagues.parquet")
playersDf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92311 entries, 0 to 92310
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        92311 non-null  object
 1   queue         92311 non-null  object
 2   tier          92311 non-null  object
 3   div           92311 non-null  object
 4   leagueId      92311 non-null  object
 5   summonerId    92311 non-null  object
 6   wins          92311 non-null  int64 
 7   losses        92311 non-null  int64 
 8   leaguePoints  92311 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 7.0+ MB


In [7]:
print(len(playersDf.iloc[40]["summonerId"]))
print(len(playersDf[playersDf["tier"]=="GOLD"]))

47
10455


0 RANKED_SOLO_5x5


In [22]:
import tqdm
# request puuid, search for matches, process match timeline data
puuids = list()
for i, entry in tqdm.tqdm(playersDf.iterrows()):
    time.sleep(120./99.)
    try:
        summonerInfo = lolWatcher.summoner.by_id(region=entry["region"], encrypted_summoner_id=str(entry["summonerId"]))
        puuids.append(summonerInfo["puuid"])
    except Exception as e:
        print("Warning, failed entry: ", e)
        puuids.append(None)
        
playersDf["puuid"] = puuids
playersDf.to_parquet("./playerLeagues-wPuuid.parquet", index=True)


2it [00:04,  2.22s/it]

Warning, failed entry:  502 Server Error: Bad Gateway for url: https://kr.api.riotgames.com/lol/summoner/v4/summoners/bCUPUTil4sEZC03DaaWho6x6ePU-y28MXtGO8p6KgjjzpCJPSsIO5gUi3w


4it [00:07,  1.74s/it]

Warning, failed entry:  502 Server Error: Bad Gateway for url: https://kr.api.riotgames.com/lol/summoner/v4/summoners/rtoc7_UY3QUs5bTCIAg7UsFRwMcmEjuEDHolmuGz_lMToDs


52it [01:17,  1.38s/it]

...

Warning, failed entry:  502 Server Error: Bad Gateway for url: https://kr.api.riotgames.com/lol/summoner/v4/summoners/cOieNFPDTccf0b0TPJ-UFOG8YKxRy_PzFBpukOIh4yMo9owq


357it [08:39,  1.44s/it]

Warning, failed entry:  502 Server Error: Bad Gateway for url: https://kr.api.riotgames.com/lol/summoner/v4/summoners/r7jUOBDxz9K9broEI3iLoQhp1dbfESf8RzFrWKr5RF8pJcc


441it [10:38,  1.45s/it]


KeyboardInterrupt: 